In [2]:
import importlib

In [15]:
import hnsw
importlib.reload(hnsw)
from hnsw import HNSW
from hnsw import heuristic
import numpy as np
from tqdm import tqdm
from datasets import load_sift_dataset, calculate_recall

In [30]:
import merge_hnsw
importlib.reload(merge_hnsw)
from merge_hnsw import merge_naive

In [31]:
distance_count = 0
def l2_distance(a, b):
    global distance_count
    distance_count+=1
    return np.linalg.norm(a - b)

In [26]:
hnsw_a = HNSW( distance_func=l2_distance, m=16, m0=32, ef=32, ef_construction=32,  neighborhood_construction = heuristic)
hnsw_b = HNSW( distance_func=l2_distance, m=16, m0=32, ef=32, ef_construction=32,  neighborhood_construction = heuristic)

In [27]:
hnsw_a.load('save/sift1m/hnsw_a.txt')
hnsw_b.load('save/sift1m/hnsw_b.txt')

In [28]:
merged_data = hnsw_a.data.copy()
merged_data.update(hnsw_b.data)

In [32]:
%%time
distance_count = 0
hnsw_merged_naive = merge_naive(hnsw_a, hnsw_b, merged_data, merge_ef = 20)
print(f'Numer of distance calculated during the merge: {distance_count}') # 696869746

Merging level: 0


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500000/500000 [5:18:03<00:00, 26.20it/s]


Merging level: 1


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 31358/31358 [09:20<00:00, 55.97it/s]


Merging level: 2


...

Merging level: 3


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 124/124 [00:00<00:00, 311.76it/s]


Merging level: 4


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 1411.67it/s]


Numer of distance calculated during the merge: 4812582791
CPU times: user 6h 30min 27s, sys: 1h 1min 16s, total: 7h 31min 43s
Wall time: 6h 50min 49s


In [33]:
hnsw_merged_naive.save(f'save/sift1m/hnsw_merged_naive_ef{merge_ef}.txt')

In [33]:
print(f'Numer of distance calculated during the merge: {distance_count}') # m=5 m0=7: 960554605; m=16, m0=32: 4812582791

Numer of distance calculated during the merge: 4812582791


In [35]:
4812582791 / 947762326

5.077837195018448

In [25]:
# merge2 m=16, m0=32 1181782572 

1.2303127441672095

In [36]:
_, test_data, groundtruth_data = load_sift_dataset(train_file = None,
                                                      test_file='../datasets/sift1m-128d/sift_query.fvecs',
                                                      groundtruth_file='../datasets/sift1m-128d/sift_groundtruth.ivecs')

In [37]:
search_ef = 32
distance_count = 0
recall, _ = calculate_recall(hnsw_merged_naive, test_data, groundtruth=groundtruth_data, k=5, ef=search_ef)
print(f'{search_ef},{recall}, {distance_count/len(test_data) }') 

Calculating recall...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [02:03<00:00, 81.09it/s]

32,0.9148400000000001, 950.5332


In [38]:
search_ef = 40
distance_count = 0
recall, _ = calculate_recall(hnsw_merged_naive, test_data, groundtruth=groundtruth_data, k=5, ef=search_ef)
print(f'{search_ef},{recall}, {distance_count/len(test_data) }') 

Calculating recall...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [02:35<00:00, 64.20it/s]

40,0.9342, 1098.4587


In [39]:
search_ef = 64
distance_count = 0
recall, _ = calculate_recall(hnsw_merged_naive, test_data, groundtruth=groundtruth_data, k=5, ef=search_ef)
print(f'{search_ef},{recall}, {distance_count/len(test_data) }') 

Calculating recall...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [04:12<00:00, 39.57it/s]

64,0.9626600000000001, 1522.9343


In [42]:
efs=[50,72]
# efs=[32,40,50,64,72]
for ef in efs:
    distance_count = 0
    recall, _ = calculate_recall(hnsw_merged_naive, test_data, groundtruth=groundtruth_data, k=5, ef=ef)
    print(f'ef={ef} recall: {recall}, avg dist: {distance_count/len(test_data) }') 

Calculating recall...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [03:21<00:00, 49.74it/s]


ef=50 recall: 0.9499400000000001, avg dist: 1278.8071
Calculating recall...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [05:57<00:00, 27.99it/s]

ef=72 recall: 0.9677400000000002, avg dist: 1659.3787
